# Data fusion for Peru

## **1.** Authentications and imports

In [ ]:
# insert your desired path to work on
import os
from os.path import join
project_path = os.path.dirname(os.getcwd())
os.chdir(join('..','..','data'))
os.getcwd()

Import necessary libraries.

In [ ]:
!pip install unidecode --quiet
from unidecode import unidecode
from functools import reduce
import numpy as np
import glob
import pandas as pd
import pprint
import seaborn as sns
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
pp = pprint.PrettyPrinter(indent=4)
%matplotlib inline

Set folder structure.

In [ ]:
config = {
    'main_brazil': 'Brazil',
    'main_peru': 'Peru'
}

# List comprehension for the folder structure code
[os.makedirs(val, exist_ok=True) for key, val in config.items()]

## **2.** Load and clean data

### 2.0. Regions data
We're only interested in the following departments: Loreto, Madre de Dios, Piura, San Martin, Tumbes and Ucayali

In [ ]:
df_regions = pd.read_csv(join(config['main_peru'], "Peru_DepID_ProvID.csv"))\
                .drop(columns=['ADM0_EN', 'ADM0_PCODE', 'Shape_Area'], axis=1)

df_regions['ADM2_ES'] = df_regions['ADM2_ES'].apply(lambda x: unidecode(x.strip()))

# Filter to Loreto, Madre de Dios, Piura, San Martin, Tumbes and Ucayali
DOI = ['PE16', 'PE17', 'PE20', 'PE22', 'PE24', 'PE25']
df_regions = df_regions[df_regions["ADM1_PCODE"].isin(DOI)].reset_index(drop=True)
df_regions

### 2.1. Population data from Census2017

In [ ]:
census2017_data = join(config['main_peru'], "Population_yearly_Province_Age_Census2017.xlsx")
census_df = pd.read_excel(census2017_data, engine='openpyxl', skiprows = 5, skipfooter = 5, usecols = "B:W", converters={'Código': str})\
                .dropna()\
                .drop(columns=['Provincia'])\
                .rename(columns={'Código':'ADM2_PCODE'})
census_df['ADM2_PCODE'] = 'PE'+census_df['ADM2_PCODE'].str.zfill(4)

census_df['PopTotal'] = census_df.iloc[:, 1:].sum(axis=1).astype(int)
census_df['Pop0_19'] = census_df.iloc[:, 1:5].sum(axis=1).astype(int)
census_df['Pop20_99'] = census_df['PopTotal'] - census_df['Pop0_19']

census_df = census_df[['ADM2_PCODE', 'PopTotal', 'Pop0_19', 'Pop20_99']]
census_df = census_df.merge(df_regions[['ADM2_PCODE']], how='right')
census_df

In [ ]:
areas = pd.read_excel(join(config['main_peru'], "socio-economic", "reporte_RurUrb.xlsx"), skiprows = 357, usecols='B:D', names=['Provincia', 'PopTotal_Urban', 'PopTotal_Rural']).replace('-', 0).dropna()
areas.drop(areas.loc[areas['Provincia'].str.contains('Total')].index, inplace=True)
areas['PopTotal_Urban'] = round(areas['PopTotal_Urban']).astype(int)
areas['PopTotal_Rural'] = round(areas['PopTotal_Rural']).astype(int)

areas['ADM2_ES'] = areas["Provincia"].str.rsplit(': ', expand = True)[1].fillna('Callao').apply(lambda x: unidecode(x.strip()))
areas = areas.merge(df_regions[['ADM2_ES','ADM2_PCODE']], on=['ADM2_ES'], how='right')[['ADM2_PCODE', 'PopTotal_Urban', 'PopTotal_Rural']]
areas

In [ ]:
pop_df = census_df.merge(areas, on=['ADM2_PCODE'], how='left')
pop_df

In [ ]:
assert all(pop_df['PopTotal'] == pop_df['Pop0_19'] + pop_df['Pop20_99'])
try:
    assert all(pop_df['PopTotal'] == pop_df['PopTotal_Urban'] + pop_df['PopTotal_Rural'])
except AssertionError:
    aux = pop_df.copy()
    aux['diff'] = pop_df['PopTotal'] - (aux['PopTotal_Urban'] + aux['PopTotal_Rural'])
    print(aux)
    print('Maximum absolute difference:', max(abs(aux['diff'])))

In [ ]:
popByDep_df = pop_df.copy()
popByDep_df = popByDep_df.merge(df_regions[['ADM1_PCODE', 'ADM2_PCODE']], on='ADM2_PCODE')
popByDep_df = popByDep_df.groupby('ADM1_PCODE').sum()
popByDep_df.reset_index(inplace=True)
popByDep_df

### 2.2. Dengue data

In [ ]:
dengue_data = join(config['main_peru'], "Dengue_cases_weekly_District_2010_2020.csv")
dengue_df = pd.read_csv(dengue_data, converters = {'Cd Depa': str}).rename(columns={'Ano': 'Year', 'Semana': 'Week', 'Cd Depa': 'ADM1_PCODE', 'Number of Records': 'Cases'})
dengue_df["ADM1_PCODE"] = 'PE'+dengue_df["ADM1_PCODE"].str.zfill(2)

dengue_df['YearWeek'] = dengue_df['Year'].astype(str) + dengue_df['Week'].astype(str).apply(lambda x: x.zfill(2))
dengue_df['YearWeek'] = dengue_df['YearWeek'].astype(int)
dengue_df['LastDayWeek'] = pd.to_datetime((dengue_df['YearWeek']-1).astype(str) + "6", format="%Y%U%w")
dengue_df.insert(loc=4, column='Month', value=pd.DatetimeIndex(dengue_df['LastDayWeek']).month)

dengue_popTotal = dengue_df[['ADM1_PCODE', 'Year', 'Month', 'Cases']]\
                    .groupby(['ADM1_PCODE', 'Year', 'Month'])\
                    .agg({'Cases': 'sum'})\
                    .reset_index()\
                    .rename(columns={'Cases':'cases_total'})

dengue_pop0_19 = dengue_df[dengue_df.Edad1 <= 19].reset_index(drop=True)[['ADM1_PCODE', 'Year', 'Month', 'Cases']]\
                                                 .groupby(['ADM1_PCODE', 'Year', 'Month'])\
                                                 .agg({'Cases': 'sum'})\
                                                 .reset_index()\
                                                 .rename(columns={'Cases':'cases0_19'})

dengue_pop20_99 = dengue_df[dengue_df.Edad1 > 19].reset_index(drop=True)[['ADM1_PCODE', 'Year', 'Month', 'Cases']]\
                                                 .groupby(['ADM1_PCODE', 'Year', 'Month'])\
                                                 .agg({'Cases': 'sum'})\
                                                 .reset_index()\
                                                 .rename(columns={'Cases':'cases20_99'})

dengue = pd.merge(dengue_popTotal, dengue_pop0_19, on=['ADM1_PCODE', 'Year', 'Month'], how='left')\
           .merge(dengue_pop20_99, on=['ADM1_PCODE', 'Year', 'Month'], how='left')

dengue = dengue[dengue["ADM1_PCODE"].isin(DOI)]\
                .reset_index(drop=True)\
                .fillna(0)\
                .astype({'cases0_19':int, 'cases20_99':int})
dengue

In [ ]:
assert all(dengue['cases_total'] == dengue['cases0_19']+dengue['cases20_99'])

### 2.3. Forest/land data

#### 2.3.1. Elevation data

In [ ]:
elev_df = pd.read_csv(join(config['main_peru'], "Elevation_stats_by_province.csv"))\
            .drop(['system:index','ADM1_ES','ADM2_ES','.geo','Province_Area'], axis=1)
elev_df

#### 2.3.2. Urbal/Rural data

In [ ]:
urbRur_df = pd.read_csv(join(config['main_peru'], "Urban_Rural_stats_by_department.csv"))\
              .rename(columns={'system:index':'ADM1_PCODE', 'Urban%':'Urban_Cover_Percent'})\
                [['ADM1_PCODE','Urban_Cover_Percent']]
urbRur_df

#### 2.3.3. Forest Cover data

In [ ]:
forest_stats = pd.read_csv(join(config['main_peru'], "Forest_loss_stats_by_department.csv")).rename(columns={'system:index': 'ADM1_PCODE'}).fillna(0)
fCover_df = forest_stats[['ADM1_PCODE']].copy()
perc_cols = forest_stats.columns[1:-4]

cnt=0
for col in perc_cols:
    cumulative_loss = forest_stats[col] if cnt == 0 else cumulative_loss + forest_stats[col]
    fCover_df['Forest_Cover_Percent_%s' %(col)] = ((forest_stats["ForestCover2000"] - cumulative_loss) / forest_stats["Department_Area"]) * 100
    cnt+=1

fCover_df

In [ ]:
depArea_df = forest_stats[['ADM1_PCODE','Department_Area']].copy().rename(columns={'Department_Area':'area_km2'})

# convert m2 to km2
depArea_df['area_km2'] = depArea_df['area_km2'].div(10**6, axis=0)
depArea_df

In [ ]:
years = np.arange(2001, 2021)
newDict={'Year':[], 'ADM1_PCODE':[], 'Forest_Cover_Percent':[]}

for id, row in fCover_df.iterrows():
    for year in years:
        newDict['Year'].append(year)
        newDict['ADM1_PCODE'].append(row.ADM1_PCODE)
        newDict['Forest_Cover_Percent'].append(row['Forest_Cover_Percent_%i' %year])

fCover_df2 = pd.DataFrame(newDict)
fCover_df2

### 2.4. Climate data

In [ ]:
era5land = pd.read_csv(join(config['main_peru'], 'ERA5land_NDVI_monthly_provinces_Peru.csv'))
era5land['Date'] = pd.to_datetime(era5land['Date'])
era5land.head()

In [ ]:
era5land.isna().sum()

In [ ]:
era5land_df = era5land.merge(elev_df, on=['ADM2_PCODE'], how='left')
era5land_df.head()

In [ ]:
era5land_df.info()

### 2.5. Social data

#### 2.5.1. Water Supply

In [ ]:
water = pd.read_excel(join(config['main_peru'], "socio-economic", "reporte_AbastacimientoAgua.xlsx"), skiprows = 405, usecols='B:K', names=['ADM1_ES', 'WaterSupply_PublicNetworkInside', 'WaterSupply_PublicNetworkBuilding', 'WaterSupply_PoolPublicUse', 'WaterSupply_Tanker', 'WaterSupply_Well', 'WaterSupply_Aqueduct', 'WaterSupply_RiverLake', 'WaterSupply_Other', 'WaterSupply_Closeby']).replace('-', 0).dropna()
water = water.astype({"WaterSupply_PublicNetworkInside":int, "WaterSupply_PublicNetworkBuilding":int, "WaterSupply_PoolPublicUse":int, "WaterSupply_Tanker":int, "WaterSupply_Well":int, "WaterSupply_Aqueduct":int, "WaterSupply_RiverLake":int, "WaterSupply_Other":int, "WaterSupply_Closeby":int})
water['ADM1_ES'] = water["ADM1_ES"].str.rsplit(', ', expand = True)[0].apply(lambda x: unidecode(x.strip()))
water = water[~water['ADM1_ES'].str.contains("Total")].reset_index(drop=True)
water = water.groupby(['ADM1_ES']).agg(sum)
water.reset_index(level=water.index.names, inplace=True)

water_df = water.merge(df_regions[['ADM1_ES', 'ADM1_PCODE']].drop_duplicates(), on=['ADM1_ES']).drop(columns=['ADM1_ES'])
cols = list(water_df)
cols.insert(0, cols.pop(cols.index('ADM1_PCODE')))
water_df = water_df.loc[:, cols]
water_df

#### 2.5.2. Electricity

In [ ]:
electricity = pd.read_excel(join(config['main_peru'], "socio-economic", "reporte_AlumbradoElectrico.xlsx"), skiprows = 405, usecols='B:D', names=['ADM1_ES', 'WithElectricity', 'WithoutElectricity']).replace('-', 0).dropna()
electricity = electricity.astype({'WithElectricity': int, 'WithoutElectricity':int})

electricity['ADM1_ES'] = electricity["ADM1_ES"].str.rsplit(', ', expand = True)[0].apply(lambda x: unidecode(x.strip()))
electricity = electricity[~electricity['ADM1_ES'].str.contains("Total")].reset_index(drop=True)
electricity = electricity.groupby(['ADM1_ES']).agg(sum)
electricity.reset_index(level=electricity.index.names, inplace=True)

electricity_df = electricity.merge(df_regions[['ADM1_ES', 'ADM1_PCODE']].drop_duplicates(), on=['ADM1_ES']).drop(columns=['ADM1_ES'])
electricity_df

#### 2.5.3. Urban/Rural Areas

In [ ]:
areas = pd.read_excel(join(config['main_peru'], "socio-economic", "reporte_RurUrb.xlsx"), skiprows = 357, usecols='B:D', names=['ADM1_ES', 'Urban', 'Rural']).replace('-', 0).dropna()
areas = areas.astype({'Urban': int, 'Rural':int})

areas.loc[areas.Urban > 0, 'Urban'] = 1
areas.loc[areas.Rural > 0, 'Rural'] = 1

areas.insert(loc=3, column='rur_prov', value=0)
areas.loc[(areas['Rural'] == 1) & (areas['Urban'] == 0), 'rur_prov'] = 1 

areas.insert(loc=4, column='urb_prov', value=0)
areas.loc[(areas['Rural'] == 0) & (areas['Urban'] == 1), 'urb_prov'] = 1 

areas.insert(loc=5, column='urbrur_prov', value=0)
areas.loc[(areas['Rural'] == 1) & (areas['Urban'] == 1), 'urbrur_prov'] = 1

areas.insert(loc=6, column='Total', value=1)

areas['ADM1_ES'] = areas["ADM1_ES"].str.rsplit(', ', expand = True)[0].apply(lambda x: unidecode(x.strip()))

areas = areas[~areas['ADM1_ES'].str.contains("Total")].reset_index(drop=True)
areas = areas.groupby(['ADM1_ES']).agg(sum)
areas.reset_index(level=areas.index.names, inplace=True)

areas['rur_prov'] /= areas['Total']
areas['rur_prov'] = areas['rur_prov'].round(3)
areas['urb_prov'] /= areas['Total']
areas['urb_prov'] = areas['urb_prov'].round(3)
areas['urbrur_prov'] /= areas['Total']
areas['urbrur_prov'] = areas['urbrur_prov'].round(3)

areas.drop(columns=['Urban', 'Rural', 'Total'], inplace=True)
areas_df = areas.merge(df_regions[['ADM1_ES', 'ADM1_PCODE']].drop_duplicates(), on=['ADM1_ES']).drop(columns=['ADM1_ES'])
areas_df

#### 2.5.4. Hygienic Service

In [ ]:
hygienic = pd.read_excel(join(config['main_peru'], "socio-economic", "reporte_ServicioHigienico.xlsx"), skiprows = 405, usecols='B:J', names=['ADM1_ES', 'Hygienic_PublicNetworkInside', 'Hygienic_PublicNetworkBuilding', 'Hygienic_SepticTank', 'Hygienic_Latrine', 'Hygienic_Well', 'Hygienic_RiverLake', 'Hygienic_OpenField', 'Hygienic_Other']).replace('-', 0).dropna()
hygienic = hygienic.astype({'Hygienic_PublicNetworkInside':int, 'Hygienic_PublicNetworkBuilding':int, 'Hygienic_SepticTank':int, 'Hygienic_Latrine':int, 'Hygienic_Well':int, 'Hygienic_RiverLake':int, 'Hygienic_OpenField':int, 'Hygienic_Other':int})

hygienic['ADM1_ES'] = hygienic["ADM1_ES"].str.rsplit(', ', expand = True)[0].apply(lambda x: unidecode(x.strip()))
hygienic = hygienic[~hygienic['ADM1_ES'].str.contains("Total")].reset_index(drop=True)
hygienic = hygienic.groupby(['ADM1_ES']).agg(sum)
hygienic.reset_index(level=hygienic.index.names, inplace=True)

hygienic_df = hygienic.merge(df_regions[['ADM1_ES', 'ADM1_PCODE']].drop_duplicates(), on=['ADM1_ES']).drop(columns=['ADM1_ES'])
hygienic_df

#### 2.5.5. Type of House

In [ ]:
house = pd.read_excel(join(config['main_peru'], "socio-economic", "reporte_TipoVivienda.xlsx"), skiprows = 355, usecols='B:K', names=['ADM1_ES', 'HouseType_Independent', 'HouseType_Flat', 'HouseType_Farm', 'HouseType_HoodAlley', 'HouseType_Hut', 'HouseType_Improvised', 'HouseType_NonHumanHabitation', 'HouseType_Other', 'HouseType_Collective']).replace('-', 0).dropna()
house = house.astype({'HouseType_Independent':int, 'HouseType_Flat':int, 'HouseType_Farm':int, 'HouseType_HoodAlley':int, 'HouseType_Hut':int, 'HouseType_Improvised':int, 'HouseType_NonHumanHabitation':int, 'HouseType_Other':int, 'HouseType_Collective':int})
house['ADM1_ES'] = house["ADM1_ES"].str.rsplit(', ', expand = True)[0].apply(lambda x: unidecode(x.strip()))

house = house[~house['ADM1_ES'].str.contains("Total")].reset_index(drop=True)
house = house.groupby(['ADM1_ES']).agg(sum)
house.reset_index(level=house.index.names, inplace=True)

house_df = house.merge(df_regions[['ADM1_ES', 'ADM1_PCODE']].drop_duplicates(), on=['ADM1_ES']).drop(columns=['ADM1_ES'])
house_df

#### 2.5.6. Merge all social-economic data

In [ ]:
dfs = [water_df, electricity_df, areas_df, hygienic_df, house_df]
social_df = reduce(lambda left, right: pd.merge(left, right, on=['ADM1_PCODE'], how='outer'), dfs)
social_df

## **3.** Create final dataframe

### 3.1. Merge all data

In [ ]:
df = pd.merge(era5land_df, pop_df[['ADM2_PCODE', 'PopTotal']], on='ADM2_PCODE', how='inner')
df

In [ ]:
df.isnull().sum()

Create population-weighted monthly averages of climatic variables.

In [ ]:
vars = ['NDVI', 'dewpoint_temperature_2m', 'humidity',
        'max_temperature_2m', 'min_temperature_2m', 'surface_pressure',
        'temperature_2m', 'total_precipitation', 
        'u_component_of_wind_10m', 'v_component_of_wind_10m',
        'max_elevation', 'mean_elevation', 'min_elevation',
        'stdDev_elevation', 'variance_elevation']

df.loc[:, vars] = df.loc[:, vars].multiply(df['PopTotal'], axis=0)
df = df.rename(columns={col: col+'_d' for col in df.columns if col in vars})
df.head()

In [ ]:
df1 = df.groupby(['Date','ADM1_PCODE'], as_index=False).sum()
df1.head()

In [ ]:
cols = ['NDVI_d', 'dewpoint_temperature_2m_d', 'humidity_d',
        'max_temperature_2m_d', 'min_temperature_2m_d', 'surface_pressure_d',
        'temperature_2m_d', 'total_precipitation_d',
        'u_component_of_wind_10m_d', 'v_component_of_wind_10m_d',
        'max_elevation_d', 'mean_elevation_d', 'min_elevation_d',
        'stdDev_elevation_d', 'variance_elevation_d']

df1.loc[:, cols] = df1.loc[:, cols].div(df1['PopTotal'], axis=0)
df1.head()

In [ ]:
df1 = df1.drop(columns=['PopTotal'], axis=1)
df1.insert(loc=1, column='Year', value=df1['Date'].dt.year)
df1.insert(loc=2, column='Month', value=df1['Date'].dt.month)
df1

In [ ]:
df2 = pd.merge(df1, popByDep_df, on=['ADM1_PCODE'], how='left')\
        .merge(depArea_df, on=['ADM1_PCODE'], how='left')\
        .merge(dengue, on=['ADM1_PCODE', 'Year', 'Month'], how='left')\
        .merge(fCover_df2, on=['ADM1_PCODE', 'Year'], how='left')\
        .merge(urbRur_df, on=['ADM1_PCODE'], how='left')\
        .merge(social_df, on=['ADM1_PCODE'], how='left')\
        .fillna(0).astype({'cases_total':int, 'cases0_19':int, 'cases20_99':int})\
        .sort_values(by=['ADM1_PCODE','Date'])
       
cols = list(df2)
cols.insert(4, cols.pop(cols.index('area_km2')))
df2 = df2.loc[:, cols]
df2

In [ ]:
df2.isna().sum()

In [ ]:
df2.to_csv(join(config['main_peru'], 'Peru_Departments_dengue_monthly.csv'), index=False)